<a href="https://colab.research.google.com/github/AliAkbarBadri/graph-centrality/blob/main/All_subgraphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [55]:
import networkx as nx
import glob
import pandas as pd

# Functions

In [5]:
def count_subgraph(G, v):
  neighbours = G.edges(v)
  if len(neighbours) == 0:
    return 1
  else:
    for e in neighbours:
      G_temp = G.copy()
      G_temp.remove_edge(*e)
      a = count_subgraph(G_temp, v)
      b = (2**G.number_of_edges(*e) - 1)*count_subgraph(nx.contracted_edge(G, e, self_loops=False), e[0])
      return a + b

In [3]:
def count_tree(T, v):
  neighbours = T.edges(v)
  if len(neighbours) == 0: 
    return 1
  else:
    for e in neighbours:
      T_temp = T.copy()
      T_temp.remove_edge(*e)
      return count_tree(T_temp, v) * (count_tree(T_temp,e[1]) + 1) 

In [6]:
line = nx.path_graph(4)
print(line.edges())

# result must be equal to 4
print(count_subgraph(line,0))
print(count_tree(line, 0))

[(0, 1), (1, 2), (2, 3)]
4
4


In [7]:
star = nx.star_graph(4)
print(star.edges())
# result must be equal to 16
print(count_subgraph(star,0))
print(count_tree(star, 0))

[(0, 1), (0, 2), (0, 3), (0, 4)]
16
16


# Get Datasets

In [48]:
! curl -L https://api.github.com/repos/AliAkbarBadri/graph-centrality/tarball --output repo.tar
! tar xf repo.tar --wildcards "*/data/*.csv" --strip-components=1
! rm -rf repo.tar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  414k  100  414k    0     0   888k      0 --:--:-- --:--:-- --:--:-- 3091k


In [68]:
for file_name in sorted(glob.glob('data/*.csv')):
  print(file_name.split("/")[1].split(".csv")[0])

email-dnc-corecipient
facebook-socfb-Amherst41
soc-dolphins
soc-wiki-Vote
web-edu
web-polblogs
web-spam


# Test on datasets

In [80]:
def create_graph_from_csv(dir_name:str) -> dict:
  graphs = {}
  for file_name in sorted(glob.glob(dir_name+'/*.csv')):
    df = pd.read_csv(file_name,header=None,names=['src', 'dest'])
    G = nx.from_pandas_edgelist(df,source="src",target="dest", create_using=nx.DiGraph())
    dataset_name = file_name.split("/")[1].split(".csv")[0]
    print(dataset_name, G.number_of_nodes(), G.number_of_edges())
    graphs[dataset_name] = G
  return graphs

In [81]:
graphs = create_graph_from_csv(dir_name= "data")

email-dnc-corecipient 906 12085
facebook-socfb-Amherst41 2235 90953
soc-dolphins 62 159
soc-wiki-Vote 889 2914
web-edu 3031 6474
web-polblogs 643 2280
web-spam 4767 37375


In [ ]:
# def get_all_subgraph_and_tree(G):
#   result = {}
#   for node in G.nodes()
#     a = count_subgraph(G, node)
#     b = count_tree(G, node)
#     result[node] = ((a,),(b,))

In [94]:
for key, value in graphs.items():
  key = "soc-dolphins"
  G = graphs[key]
  print(key, G.number_of_nodes(), G.number_of_edges())
  
  for node in G.nodes():
    
  
  
  break


email-dnc-corecipient 906 12085
facebook-socfb-Amherst41 2235 90953
soc-dolphins 62 159
soc-wiki-Vote 889 2914
web-edu 3031 6474
web-polblogs 643 2280
web-spam 4767 37375


## soc-dolphins

In [92]:

count_subgraph(G, 15)

4

In [ ]:
count_subgraph()

## soc-wiki-Vote

In [ ]:
G = graphs['soc-wiki-Vote'] 
print(G.number_of_nodes(), G.number_of_edges())

## web-polblogs

In [ ]:
G = graphs['soc-wiki-Vote'] 
print(G.number_of_nodes(), G.number_of_edges())

## web-edu

In [ ]:
G = graphs['soc-wiki-Vote'] 
print(G.number_of_nodes(), G.number_of_edges())

## email-dnc-corecipient

In [ ]:
G = graphs['soc-wiki-Vote'] 
print(G.number_of_nodes(), G.number_of_edges())

## web-spam

In [ ]:
G = graphs['soc-wiki-Vote'] 
print(G.number_of_nodes(), G.number_of_edges())

## facebook-socfb-Amherst41